# Dynamic WSCC 9-bus System with Switch

In [ ]:
import requests
import glob

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

url = 'https://git.rwth-aachen.de/acs/public/grid-data/cim-grid-data/-/raw/master/WSCC-09/WSCC-09_RX_Dyn/WSCC-09_RX'
filename = 'WSCC-09'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')

files = glob.glob(filename+'_*.xml')
print(files)

In [ ]:
import villas.dataprocessing.plottools as pt
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import dpsimpy
import numpy as np

In [ ]:
sim_name = 'WSCC-9bus_dyn_switch'

## Simulation

In [ ]:
dpsimpy.Logger.set_log_dir('logs/' + sim_name)
reader = dpsimpy.CIMReader(sim_name, dpsimpy.LogLevel.debug, dpsimpy.LogLevel.debug)
system = reader.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.TransientStability)

# Extend topology with switch
sw = dpsimpy.dp.ph1.Switch('Fault', dpsimpy.LogLevel.info)
sw.set_parameters(1e9, 0.1)
sw.connect([dpsimpy.dp.SimNode.gnd, system.node('BUS6')])
sw.open()
system.add(sw)

# Use omegNom for torque conversion in SG models for validation with PSAT
system.component('GEN1').set_model_flags(False, False)
system.component('GEN2').set_model_flags(False, False)
system.component('GEN3').set_model_flags(False, False)

logger = dpsimpy.Logger(sim_name)

for i in range(1, 10):
    logger.log_attribute('v' + str(i), 'v', system.node('BUS' + str(i)))

for i in range(1, 4):
    logger.log_attribute('wr_' + str(i), 'w_r', system.component('GEN' + str(i)))
    logger.log_attribute('delta_r_' + str(i), 'delta_r', system.component('GEN' + str(i)))
    logger.log_attribute('P_elec_' + str(i), 'P_elec', system.component('GEN' + str(i)))
    logger.log_attribute('P_mech_' + str(i), 'P_mech', system.component('GEN' + str(i)))


sim = dpsimpy.Simulation(sim_name, dpsimpy.LogLevel.debug)
sim.set_system(system)
sim.set_time_step(0.0001)
sim.set_final_time(2.0)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_solver(dpsimpy.Solver.MNA)
sim.do_init_from_nodes_and_terminals(True)

sw_event_1 = dpsimpy.event.SwitchEvent(0.2, sw, True)
sim.add_event(sw_event_1)

sim.add_logger(logger)
sim.run()


## Read simulation results

In [ ]:
path = 'logs/WSCC-9bus_dyn_switch/'
logName = 'WSCC-9bus_dyn_switch'
logFilename = path + logName + '.csv'
print(logFilename)

ts_dpsim = rt.read_timeseries_dpsim(logFilename)
phasors = ts.phasors(ts_dpsim)

### Phasors at last time step in per unit

In [ ]:
nominal_voltages = {'v1': 16500, 'v2': 18000, 'v3': 13800, 
                         'v4': 230000, 'v5': 230000, 'v6': 230000, 
                         'v7': 230000, 'v8': 230000, 'v9': 230000} 
for node, nom_voltage in nominal_voltages.items():
    print(node + ': ' + str(phasors[node]['abs'].values[0] / nom_voltage) + '<' + str(phasors[node]['phase'].values[0]))

### Plot node voltages

In [ ]:
plt.figure()
plt.plot(phasors['v1']['abs'].time, phasors['v1']['abs'].values, label='v1.abs')
plt.plot(phasors['v2']['abs'].time, phasors['v2']['abs'].values, label='v2.abs')
plt.plot(phasors['v3']['abs'].time, phasors['v3']['abs'].values, label='v3.abs')
plt.legend()
plt.ylim([0,20000])

plt.figure()
plt.plot(phasors['v4']['abs'].time, phasors['v4']['abs'].values, label='v4.abs')
plt.plot(phasors['v5']['abs'].time, phasors['v5']['abs'].values, label='v5.abs')
plt.plot(phasors['v6']['abs'].time, phasors['v6']['abs'].values, label='v6.abs')
plt.plot(phasors['v7']['abs'].time, phasors['v7']['abs'].values, label='v7.abs')
plt.plot(phasors['v8']['abs'].time, phasors['v8']['abs'].values, label='v8.abs')
plt.plot(phasors['v9']['abs'].time, phasors['v9']['abs'].values, label='v9.abs')
plt.legend()
plt.ylim([0,240000])

### Generator Speed

In [ ]:
plt.figure()
plt.plot(ts_dpsim['wr_1'].time, ts_dpsim['wr_1'].values)
plt.plot(ts_dpsim['wr_2'].time, ts_dpsim['wr_2'].values)
plt.plot(ts_dpsim['wr_3'].time, ts_dpsim['wr_3'].values)
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (rad/s)')
#plt.savefig('wscc_9bus_fault_gen_speed.pdf')

## Validation

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/dpsim/dpsim-results/raw/master/WSCC-9bus/WSCC-9bus_dyn_switch_mod_inertia.csv'
local_file = 'reference-results/WSCC-9bus_dyn_switch.csv'
urllib.request.urlretrieve(url, local_file) 

ts_sl = rt.read_timeseries_simulink(local_file)
phasors_sl = ts.phasors(ts_sl)

In [ ]:
plt.figure()
plt.plot(phasors_sl['v1']['abs'].time, phasors_sl['v1']['abs'].values, label='v1.abs')
plt.plot(phasors_sl['v2']['abs'].time, phasors_sl['v2']['abs'].values, label='v2.abs')
plt.plot(phasors_sl['v3']['abs'].time, phasors_sl['v3']['abs'].values, label='v3.abs')
plt.legend()
plt.ylim([0,20000])

plt.figure()
plt.plot(phasors_sl['v4']['abs'].time, phasors_sl['v4']['abs'].values, label='v4.abs')
plt.plot(phasors_sl['v5']['abs'].time, phasors_sl['v5']['abs'].values, label='v5.abs')
plt.plot(phasors_sl['v6']['abs'].time, phasors_sl['v6']['abs'].values, label='v6.abs')
plt.plot(phasors_sl['v7']['abs'].time, phasors_sl['v7']['abs'].values, label='v7.abs')
plt.plot(phasors_sl['v8']['abs'].time, phasors_sl['v8']['abs'].values, label='v8.abs')
plt.plot(phasors_sl['v9']['abs'].time, phasors_sl['v9']['abs'].values, label='v9.abs')
plt.legend()
plt.ylim([0,240000])

In [ ]:
plt.figure()
plt.plot(ts_sl['wr_1'].time, ts_sl['wr_1'].values)
plt.plot(ts_sl['wr_2'].time, ts_sl['wr_2'].values)
plt.plot(ts_sl['wr_3'].time, ts_sl['wr_3'].values)
plt.xlabel('time (s)')
plt.ylabel('mechanical speed (rad/s)')

In [ ]:
import numpy as np
# assert np.all(ts_sl['wr_1'].values - ts_dpsim['wr_1'].values < 0.01)